### Juntando antiga (5%) com nova (95%) 

In [12]:
import random
import numpy as np
import pandas as pd

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
    
    # Ordenar o DataFrame pela coluna 'Ordem de Movimentação'
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        errou = 0
        pode_mover = True

        if (errou==0):
            
            # Verificar se podemos remover do porto preenchido
            if andar_ini < 1:
                if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                    errou =1
                    pode_mover = False
            
            # Verificar se podemos adicionar à balsa
            if andar_fin > 0:
                if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                    errou =1
                    pode_mover = False

            # Verifica se o lugar está ocupado
            if balsa[andar_fin , x_fin, y_fin] == 1:
                errou =1
                pode_mover = False

                    
            if pode_mover:
                # Atualizar porto_preenchido e balsa
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                cal_movimentos += (
                                ((3-andar_ini)+(3-andar_fin)) #alturas
                                + (abs(x_ini - x_fin))
                                + ((2-y_ini)+(y_fin))
                                )
            else:
                cal_movimentos += 1000
        else:
            cal_movimentos += 1000

    return cal_movimentos

def selecao_roleta(populacao, fitness):
    soma_fitness = sum(fitness)
    pick = random.uniform(0, soma_fitness)
    atual = 0
    for cromossomo, fit in zip(populacao, fitness):
        atual += fit
        if atual >= pick:
            return cromossomo

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    # Copiar o segmento
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    # Resolver conflitos para filho1
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    # Resolver conflitos para filho2
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    # Cruzamento para as posições iniciais (0-29)
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    # Cruzamento para as posições finais (30-59)
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    # Cruzamento para a ordem de movimentação (60-89)
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = selecao_roleta(populacao, fitness)
            mae = selecao_roleta(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        # Ordenando antiga geração pelo fitness e pegando os 5 melhores
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        populacao_antiga_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        melhor_5_porcento = int(tamanho_populacao * 0.05)
        melhores_5_antiga = [cromossomo for cromossomo, _ in populacao_antiga_ordenada[:melhor_5_porcento]]

        # Ordenando nova geração pelo fitness e pegando os 95 melhores
        fitness_nova_geracao = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in nova_populacao]
        populacao_nova_ordenada = sorted(zip(nova_populacao, fitness_nova_geracao), key=lambda x: x[1])
        limite_5 = tamanho_populacao - melhor_5_porcento    
        populacao = [cromossomo for cromossomo, _ in populacao_nova_ordenada[:limite_5]]

        # Juntado os dois
        populacao.extend(melhores_5_antiga)
        print("  ",len(populacao))
        # Calculando o fitness os dois juntos
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 100
geracoes = 10
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


   100
Geração 1: Melhor fitness = 12149
   100
Geração 2: Melhor fitness = 12147
   100
Geração 3: Melhor fitness = 12137
   100
Geração 4: Melhor fitness = 12137
   100
Geração 5: Melhor fitness = 12137
   100
Geração 6: Melhor fitness = 12137
   100
Geração 7: Melhor fitness = 12137
   100
Geração 8: Melhor fitness = 12137
   100
Geração 9: Melhor fitness = 12137
   100
Geração 10: Melhor fitness = 12137
Melhor cromossomo encontrado: [26, 7, 25, 4, 20, 16, 22, 13, 24, 10, 8, 1, 27, 17, 11, 18, 29, 21, 19, 0, 12, 6, 3, 23, 14, 5, 28, 9, 15, 2, 14, 18, 8, 13, 15, 12, 6, 10, 3, 4, 25, 11, 26, 5, 23, 1, 29, 17, 16, 22, 21, 9, 2, 0, 19, 27, 7, 20, 24, 28, 5, 28, 9, 15, 10, 24, 16, 25, 0, 6, 12, 2, 21, 1, 26, 8, 19, 7, 14, 23, 20, 17, 18, 3, 13, 22, 4, 11, 27, 29]
Melhor fitness: 12137


#### Verifica se ha numeros repetidos


In [4]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
24         24               22              5                      0
3           3               23              6                      1
8           8               20              3                      2
13         13               21              8                      3
22         22               26             10                      4
10         10               25              2                      5
0           0               17              1                      6
4           4               18              0                      7
23         23                6              4                      8
11         11               28             22                      9
6           6               16             14                     10
29         29               13             16                     11
25         25                2             26       

### Simular Melhor cromomossomo

In [5]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []
    

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False
        
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
            print("Estado atual do Porto Preenchido:")
            print(porto_preenchido)
            print("Estado atual da Balsa:")
            print(balsa)
            print("\n")
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 24 de (22)(22)(1, 2, 1) para (5)(5)(0, 1, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 0 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [0 0 1]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 3 de (23)(23)(1, 2, 2) para (6)(6)(0, 2, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 0 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [0 0 1]
  [1 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 8 de (20)(20)(1, 1, 2) para (3)(3)(0, 1, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 0]
  [1 0 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [1 0 1]
  [1 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  

In [ ]:
pos = 27
andar_ini, x_ini, y_ini = pos // 12, (pos % 12) // 3, (pos % 12) % 3
print(andar_ini," ",x_ini," ",y_ini)

2   1   0
